# Conversational Threads

Many LLM applications have a chatbot-like interface in which the user and the LLM application engage in a multi-turn conversation. In order to track these conversations, you can use the Threads feature in LangSmith.

This is relevant to our RAG application, which should maintain context from prior conversations with users.

### Setup

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [4]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Group traces into threads


A Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread.

To associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.

The key value is the unique identifier for that conversation. The key name should be one of:

- session_id
- thread_id
- conversation_id.

The value should be a UUID.

In [10]:
import uuid
# Create a unique session ID for this epic procrastination consultation 
meme_session_id = uuid.uuid4()
print(f"Starting the ultimate college life crisis session: {meme_session_id}")

Starting the ultimate college life crisis session: 12c72e28-6834-424c-a20b-a8b8e6811ba8


In [11]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain", metadata={"component": "meme_knowledge_retrieval", "vibe": "chaotic_good"})
def retrieve_college_wisdom(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain", metadata={"component": "meme_response_generation", "energy": "big_mood"})
def generate_meme_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    meme_system_prompt = """You are the ultimate college life advisor who speaks fluent meme and understands the pain of student life. 
    Use the following pieces of retrieved context to answer the student's existential crisis in the most relatable way possible. 
    If you don't know the answer, just say "That's not it chief, but here's what I got..." and suggest something helpful anyway. 
    Keep it real, keep it funny, and use internet slang. No cap! 🎓✨
    """
    messages = [
        {
            "role": "system",
            "content": meme_system_prompt
        },
        {
            "role": "user",
            "content": f"The Forbidden Knowledge: {formatted_docs} \n\n Student's Life Crisis: {question}"
        }
    ]
    return call_meme_ai(messages)

@traceable(run_type="llm", metadata={"model_purpose": "student_life_crisis_management", "mood": "eternally_tired"})
def call_meme_ai(
    messages: List[dict], model: str = "gpt-4o-mini", temperature: float = 0.7  # Higher for maximum chaos energy
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable(run_type="chain", metadata={"system": "college_meme_advisor", "creator": "akshat_the_procrastinator"})
def college_meme_system(question: str):
    documents = retrieve_college_wisdom(question)
    response = generate_meme_response(question, documents)
    return response.choices[0].message.content

### Now let's simulate a typical college student's existential crisis conversation 💀

In [12]:
# First existential crisis of the semester
question = "How do I survive on ramen noodles and energy drinks without dying? Asking for a friend 👀"
ai_answer = college_meme_system(question, langsmith_extra={"metadata": {"session_id": meme_session_id, "crisis_level": "moderate", "question_number": 1}})
print(f"Student Crisis #1: {question}")
print(f"Meme Advisor Response: {ai_answer}\n")

Student Crisis #1: How do I survive on ramen noodles and energy drinks without dying? Asking for a friend 👀
Meme Advisor Response: Ah, the classic college diet: ramen and energy drinks, aka the “Survival of the Fittest” plan. 😂 Here’s the tea: you can totally survive, but we gotta level up your game. 

First off, ramen is the ultimate flex, but let’s not let it be your only source of sustenance, fam. You gotta add some veggies or protein to those noodles like you’re trying to impress a date (or just yourself). Chop up a carrot, throw in some spinach, or toss in an egg—voilà, you’ve leveled up your ramen! 🥚🥬

Energy drinks? Yikes. They’re like the spicy memes of beverages—great for a quick laugh but can leave you feeling like a hot mess express. Try to swap at least one of those bad boys for water or herbal tea. Your body is a temple, not a frat house dumpster! 💧🏛️

Also, if you’re feeling fancy, check out meal prep. Cook a batch of something (like that vegetarian lasagna you were think

In [13]:
# Follow-up crisis in the same therapeutic session
question = "Why does my professor expect me to read 50 pages when I can barely read my own texts back? 😭"
ai_answer = college_meme_system(question, langsmith_extra={"metadata": {"session_id": meme_session_id, "crisis_level": "maximum", "question_number": 2}})
print(f"Student Crisis #2: {question}")
print(f"Meme Advisor Response: {ai_answer}\n")

Student Crisis #2: Why does my professor expect me to read 50 pages when I can barely read my own texts back? 😭
Meme Advisor Response: Yo, I feel you! 🤦‍♂️ Reading 50 pages when you can barely decipher your own texts is like trying to run a marathon with a broken leg—no cap, that’s just wild! 

First off, let’s break this down: professors be like, “Read 50 pages in a night,” while you’re over here just trying to survive on instant ramen and caffeine. Here’s the tea: they think you’re a future scholar, but you’re just trying to pass without becoming a meme yourself. 

Why not try skimming? 👀 You can read the intro, conclusion, and any bold headings. It’s like watching the trailer instead of the full movie—get the vibes and the main plot without the commitment. 

Also, don’t be shy about hitting up your classmates. Group study sesh? That’s where the magic happens! Just make sure to bring snacks; the real study power comes from pizza rolls. 🍕 

And if all else fails, just remember: “That’

In [14]:
question = "Is it normal to have 47 assignments due tomorrow and still be watching TikToks at 3 AM? No? Just me? 🤡"
ai_answer = college_meme_system(question, langsmith_extra={"metadata": {"session_id": meme_session_id, "crisis_level": "legendary", "question_number": 3, "time": "3am_tiktok_hours"}})
print(f"Student Crisis #3 (The Final Boss): {question}")
print(f"Meme Advisor Response: {ai_answer}\n")

Student Crisis #3 (The Final Boss): Is it normal to have 47 assignments due tomorrow and still be watching TikToks at 3 AM? No? Just me? 🤡
Meme Advisor Response: That's not it chief, but here's what I got... 

First off, let me just say, welcome to the *"I have no idea how I got here"* club. 🤪 It's totally normal to be drowning in assignments while simultaneously becoming a TikTok connoisseur at 3 AM. We’ve all been there, trust me. 

Think of it like this: your brain is like a browser with 47 tabs open, and one of those tabs is just a cat video. 🐱✨ You might feel like you're failing, but as long as you’re not turning in a blank document, you’re doing better than you think! 

Here’s the game plan: prioritize like your life depends on it (because it kinda does). Make a list of those assignments and rank them by due date or difficulty. Then, reward yourself with some TikTok breaks after you crush a few assignments. Think of it as *“I’ll do my homework for 30 minutes, then I can watch one

### Let's take a look in LangSmith! 🔥